# Read dataset

In [1]:
import pandas as pd
import json

In [3]:
# qqp_path = "data/paraphrase_identification/qqp.train.jsonl"
# qqp_path = "data/paraphrase_identification/qqp.val.jsonl"
qqp_path = "data/paraphrase_identification/qqp.dev.jsonl"
# qqp_path = "data/paraphrase_identification/paws.dev_and_test.jsonl"

In [5]:
word_overlap.head()

,id,qid1,qid2,sentence1,sentence2,is_duplicate
0,312147,436549,57965,Is it a good decision to ban currency of 500 a...,Is it a good decision to ban currency of 500 a...,1
1,74079,127018,127019,Does eating eggs also cause bird flu?,Can eating boiled egg or omelette cause bird flu?,1
2,6705,13126,13127,Can God make a boulder so heavy he can't lift it?,If God is real could he make a rock that is to...,1
3,351429,50779,293918,How do I prepare for neet 2017 in 4 months?,How can I prepare for NEET 2017 in 4 months?,1
4,256701,255449,194319,Does Trump want to lose the election? Is it po...,Does Trump really want to be president? Why is...,1


In [6]:
to_count = {}
for i in range(len(word_overlap)):
  if word_overlap['is_duplicate'][i] not in to_count:
    to_count[word_overlap['is_duplicate'][i]] = 1
  else:
    to_count[word_overlap['is_duplicate'][i]] += 1
to_count

{1: 1846, 0: 3154}

In [7]:
# for i in range(len(word_overlap)):
#   if word_overlap['gold_label'][i] == '-':
#     word_overlap = word_overlap.drop(labels=i, axis=0)

In [8]:
# word_overlap = word_overlap.reset_index()

In [9]:
_c = 0
for i in range(len(word_overlap)):
  if word_overlap['is_duplicate'][i] == '-':
    _c += 1
_c

0

In [10]:
pair_and_label = []
for i in range(len(word_overlap)):
    pair_and_label.append((word_overlap['sentence1'][i], word_overlap['sentence2'][i], 
                             word_overlap['is_duplicate'][i]))

In [11]:
len(pair_and_label)

5000

In [12]:
word_overlap['pair_label'] = pair_and_label

In [13]:
word_overlap.head()

,id,qid1,qid2,sentence1,sentence2,is_duplicate,pair_label
0,312147,436549,57965,Is it a good decision to ban currency of 500 a...,Is it a good decision to ban currency of 500 a...,1,(Is it a good decision to ban currency of 500 ...
1,74079,127018,127019,Does eating eggs also cause bird flu?,Can eating boiled egg or omelette cause bird flu?,1,"(Does eating eggs also cause bird flu?, Can ea..."
2,6705,13126,13127,Can God make a boulder so heavy he can't lift it?,If God is real could he make a rock that is to...,1,(Can God make a boulder so heavy he can't lift...
3,351429,50779,293918,How do I prepare for neet 2017 in 4 months?,How can I prepare for NEET 2017 in 4 months?,1,"(How do I prepare for neet 2017 in 4 months?, ..."
4,256701,255449,194319,Does Trump want to lose the election? Is it po...,Does Trump really want to be president? Why is...,1,(Does Trump want to lose the election? Is it p...


# Lexical Overlap

In [14]:
def lexical_overlap(pair_label):
  premise = pair_label[0]
  hypothesis = pair_label[1]
  label = pair_label[2]
  #all = {"neutral": [], "contradiction": [], "entailment": [], "else": []} 

  prem_words = []
  hyp_words = []

  for word in premise.split():
    if word not in [".", "?", "!"]:
      prem_words.append(word.lower())
  
  for word in hypothesis.split():
    if word not in [".", "?", "!"]:
      hyp_words.append(word.lower())
  
  prem_filtered = " ".join(prem_words)
  hyp_filtered = " ".join(hyp_words)

  count = 0
  for word in hyp_words:
    if word in prem_words:
      count += 1

  if count >= len(hyp_words)*80/100:
    all_in = True
  else:
    all_in = False
  
  if all_in:
    return 1
  else:
    return 0

In [15]:
lexical_overlap(word_overlap['pair_label'][0])

1

In [16]:
word_overlap['lexical_overlap'] = word_overlap['pair_label'].apply(lexical_overlap)

In [17]:
word_overlap.head()

,id,qid1,qid2,sentence1,sentence2,is_duplicate,pair_label,lexical_overlap
0,312147,436549,57965,Is it a good decision to ban currency of 500 a...,Is it a good decision to ban currency of 500 a...,1,(Is it a good decision to ban currency of 500 ...,1
1,74079,127018,127019,Does eating eggs also cause bird flu?,Can eating boiled egg or omelette cause bird flu?,1,"(Does eating eggs also cause bird flu?, Can ea...",0
2,6705,13126,13127,Can God make a boulder so heavy he can't lift it?,If God is real could he make a rock that is to...,1,(Can God make a boulder so heavy he can't lift...,0
3,351429,50779,293918,How do I prepare for neet 2017 in 4 months?,How can I prepare for NEET 2017 in 4 months?,1,"(How do I prepare for neet 2017 in 4 months?, ...",1
4,256701,255449,194319,Does Trump want to lose the election? Is it po...,Does Trump really want to be president? Why is...,1,(Does Trump want to lose the election? Is it p...,0


# Word Swapping

In [18]:
import Levenshtein

In [19]:
def all_count(pair_label):
  premise = pair_label[0]
  hypothesis = pair_label[1]
  label = pair_label[2]

  all = {"neutral": [], "contradiction": [], "entailment": [], "else": []} 
  twen = []
  prem_words = []
  hyp_words = []

  for word in premise.split():
    if word not in [".", "?", "!"]:
      prem_words.append(word.lower())
  
  for word in hypothesis.split():
    if word not in [".", "?", "!"]:
      hyp_words.append(word.lower())
  
  prem_filtered = " ".join(prem_words)
  hyp_filtered = " ".join(hyp_words)

  count = 0
  for word in hyp_words:
    if word in prem_words:
      count += 1

  if count > len(hyp_words)*0/100:
    all_in = True
  else:
    all_in = False
  
  if all_in:
    twen.append((premise, hypothesis))

  return twen

In [20]:
def detect_swapping(prem_and_hypo):
  new_prem, new_hypo = '', ''
  prem = prem_and_hypo[0].replace("  ", " ")
  hypo = prem_and_hypo[1].replace("  ", " ")
  tokens_prem = prem.lower().split()
  tokens_hypo = hypo.lower().split()

  intersect = list(set(tokens_prem).intersection(tokens_hypo))

  for i in range(len(tokens_prem)): 
    if tokens_prem[i] in intersect:
      new_prem += tokens_prem[i]+' '
  for i in range(len(tokens_hypo)):
    if tokens_hypo[i] in intersect:
      new_hypo += tokens_hypo[i]+' '
  
  if len(new_prem.split()) == 1 and len(new_hypo.split()) == 1:
    return "-"
  elif len(new_prem.split()) == 0 and len(new_hypo.split()) == 0:
    return "-"
  
  if len(new_prem.strip()) > len(new_hypo.strip()):
    dif = len(new_prem.strip())-len(new_hypo.strip())
    distance = Levenshtein.distance(new_prem.strip(), new_hypo.strip())
    if dif == distance:
      return "Not Swap"
    else:
      return "Swap"
  else:
    dif = len(new_hypo.strip())-len(new_prem.strip())
    distance = Levenshtein.distance(new_prem.strip(), new_hypo.strip())
    if dif == distance:
      return "Not Swap"
    else:
      return "Swap"

In [21]:
detect_swapping(("Carl Newton and I wrote a letter", "Carl wrote a letter"))

'Not Swap'

In [22]:
swap, not_swap, dash = 0, 0, 0
for i in word_overlap['pair_label']:
  res = detect_swapping(i)
  if res == "Swap":
    swap += 1
  elif res == "Not Swap":
    not_swap += 1
  else:
    dash += 1

In [23]:
word_overlap['word_swapping'] = word_overlap['pair_label'].apply(detect_swapping)

In [24]:
word_overlap.head()

,id,qid1,qid2,sentence1,sentence2,is_duplicate,pair_label,lexical_overlap,word_swapping
0,312147,436549,57965,Is it a good decision to ban currency of 500 a...,Is it a good decision to ban currency of 500 a...,1,(Is it a good decision to ban currency of 500 ...,1,Not Swap
1,74079,127018,127019,Does eating eggs also cause bird flu?,Can eating boiled egg or omelette cause bird flu?,1,"(Does eating eggs also cause bird flu?, Can ea...",0,Not Swap
2,6705,13126,13127,Can God make a boulder so heavy he can't lift it?,If God is real could he make a rock that is to...,1,(Can God make a boulder so heavy he can't lift...,0,Swap
3,351429,50779,293918,How do I prepare for neet 2017 in 4 months?,How can I prepare for NEET 2017 in 4 months?,1,"(How do I prepare for neet 2017 in 4 months?, ...",1,Not Swap
4,256701,255449,194319,Does Trump want to lose the election? Is it po...,Does Trump really want to be president? Why is...,1,(Does Trump want to lose the election? Is it p...,0,Not Swap


# Hypothesis Lenght

In [25]:
from sklearn import preprocessing
import numpy as np

In [26]:
def hypo_len(pair_label):
  hypo_tokens = pair_label[1].strip().split()
  return len(hypo_tokens)

In [27]:
word_overlap['hypothesis length'] = word_overlap['pair_label'].apply(hypo_len)

In [28]:
word_overlap.head()

,id,qid1,qid2,sentence1,sentence2,is_duplicate,pair_label,lexical_overlap,word_swapping,hypothesis length
0,312147,436549,57965,Is it a good decision to ban currency of 500 a...,Is it a good decision to ban currency of 500 a...,1,(Is it a good decision to ban currency of 500 ...,1,Not Swap,12
1,74079,127018,127019,Does eating eggs also cause bird flu?,Can eating boiled egg or omelette cause bird flu?,1,"(Does eating eggs also cause bird flu?, Can ea...",0,Not Swap,9
2,6705,13126,13127,Can God make a boulder so heavy he can't lift it?,If God is real could he make a rock that is to...,1,(Can God make a boulder so heavy he can't lift...,0,Swap,17
3,351429,50779,293918,How do I prepare for neet 2017 in 4 months?,How can I prepare for NEET 2017 in 4 months?,1,"(How do I prepare for neet 2017 in 4 months?, ...",1,Not Swap,10
4,256701,255449,194319,Does Trump want to lose the election? Is it po...,Does Trump really want to be president? Why is...,1,(Does Trump want to lose the election? Is it p...,0,Not Swap,13


In [29]:
length = np.array([word_overlap['hypothesis length']])
length

array([[12,  9, 17, ...,  8, 10,  9]])

In [30]:
length = np.array([word_overlap['hypothesis length']]).reshape(-1, 1)
scaler = preprocessing.MinMaxScaler()
scale = scaler.fit_transform(length)
scale

array([[0.14516129],
       [0.09677419],
       [0.22580645],
       ...,
       [0.08064516],
       [0.11290323],
       [0.09677419]])

In [31]:
scale_list = []
for i in range(len(scale)):
  scale_list.append(scale[i][0])

In [32]:
word_overlap['hypo_len'] = scale_list

In [33]:
word_overlap = word_overlap.drop("hypothesis length", axis=1)

In [34]:
word_overlap.head()

,id,qid1,qid2,sentence1,sentence2,is_duplicate,pair_label,lexical_overlap,word_swapping,hypo_len
0,312147,436549,57965,Is it a good decision to ban currency of 500 a...,Is it a good decision to ban currency of 500 a...,1,(Is it a good decision to ban currency of 500 ...,1,Not Swap,0.145161
1,74079,127018,127019,Does eating eggs also cause bird flu?,Can eating boiled egg or omelette cause bird flu?,1,"(Does eating eggs also cause bird flu?, Can ea...",0,Not Swap,0.096774
2,6705,13126,13127,Can God make a boulder so heavy he can't lift it?,If God is real could he make a rock that is to...,1,(Can God make a boulder so heavy he can't lift...,0,Swap,0.225806
3,351429,50779,293918,How do I prepare for neet 2017 in 4 months?,How can I prepare for NEET 2017 in 4 months?,1,"(How do I prepare for neet 2017 in 4 months?, ...",1,Not Swap,0.112903
4,256701,255449,194319,Does Trump want to lose the election? Is it po...,Does Trump really want to be president? Why is...,1,(Does Trump want to lose the election? Is it p...,0,Not Swap,0.161290


# Overlapping Score

In [35]:
def get_overlap_score(pair_label):
    prem_words = []
    hyp_words = []

    #   premise = pair_label[0].strip()
    #   hypothesis = pair_label[1].strip()
    #   gold_label = pair_label[2].strip()
    premise = pair_label[0].strip()
    hypothesis = pair_label[1].strip()
    gold_label = pair_label[2]
        
    for word in premise.split():
        if word not in [".", "?", "!"]:
            prem_words.append(word.lower())

    for word in hypothesis.split():
        if word not in [".", "?", "!"]:
            hyp_words.append(word.lower())

    prem_filtered = " ".join(prem_words)
    hyp_filtered = " ".join(hyp_words)

    count = 0
    for word in hyp_words:
        if word in prem_words:
            count+=1

    if len(hyp_words) != 0:
        overlap_score = count/len(hyp_words)        
    else:
        overlap_score = 0
    return overlap_score

In [36]:
word_overlap['overlapping score'] = word_overlap['pair_label'].apply(get_overlap_score)

In [37]:
word_overlap.head()

,id,qid1,qid2,sentence1,sentence2,is_duplicate,pair_label,lexical_overlap,word_swapping,hypo_len,overlapping score
0,312147,436549,57965,Is it a good decision to ban currency of 500 a...,Is it a good decision to ban currency of 500 a...,1,(Is it a good decision to ban currency of 500 ...,1,Not Swap,0.145161,1.000000
1,74079,127018,127019,Does eating eggs also cause bird flu?,Can eating boiled egg or omelette cause bird flu?,1,"(Does eating eggs also cause bird flu?, Can ea...",0,Not Swap,0.096774,0.444444
2,6705,13126,13127,Can God make a boulder so heavy he can't lift it?,If God is real could he make a rock that is to...,1,(Can God make a boulder so heavy he can't lift...,0,Swap,0.225806,0.294118
3,351429,50779,293918,How do I prepare for neet 2017 in 4 months?,How can I prepare for NEET 2017 in 4 months?,1,"(How do I prepare for neet 2017 in 4 months?, ...",1,Not Swap,0.112903,0.900000
4,256701,255449,194319,Does Trump want to lose the election? Is it po...,Does Trump really want to be president? Why is...,1,(Does Trump want to lose the election? Is it p...,0,Not Swap,0.161290,0.538462


In [38]:
word_overlap.to_csv("qqp_dev_overlapping_features.csv")

In [39]:
word_overlap.shape

(5000, 11)